# Functions and HTML

### Introduction

Click [here](http://www.espn.com/nhl/player/_/id/1569/rick-nash), and then click [here](http://www.espn.com/nhl/player/_/id/3081/henrik-lundqvist).  We see two websites that describe two different players, but each website is actually pretty similar.


<img src="https://github.com/apis-jigsaw/building-websites/blob/main/rick-nash.png?raw=1" width="50%">

<img src="https://github.com/apis-jigsaw/building-websites/blob/main/henrik-picture.png?raw=1" width="50%">

## It's pretty similar underneath

Now let's take a look at the HTML.

>Remember, we can view the HTML of a webpage by pressing `command+shift+c` or `control+shift+c` on a pc.  
> Or by going to the Chrome menu at the top, followed by `View -> Developer -> Developer Tools`.


* Here is the HTML for Rick Nash

<img src="https://github.com/apis-jigsaw/building-websites/blob/main/rick-nash-html.png?raw=1">

* And here is the HTML for Henrik Lundqvist

<img src="https://github.com/apis-jigsaw/building-websites/blob/main/henrik-html.png?raw=1">

And remember that this HTML is really just a string.  So what we can imagine our ESPN website doing is slightly changing a string based on the website that a user visits.

And really, when we mean based on the website that a user visits, we mean based on the url at the top of the page.  For example take a look at our url for Henrik Lundqvist.

<img src="https://github.com/apis-jigsaw/building-websites/blob/main/henrik-url.png?raw=1" width="50%">

And now let's look at the url for Rick Nash.

<img src="https://github.com/apis-jigsaw/building-websites/blob/main/nash-url.png?raw=1" width="50%">

So we want something that can take in different user inputs and from there generate slightly different strings based on this input.

This seems like a job for functions.  Let's see how.

### Generating HTML with Functions

Let's start with the following function, `playerHeading()`.  This function takes in a player's name and from there generates a heading with the player's name.

In [ ]:
def playerHeading(playerName):
    return f"<h1> {playerName} </h1>"

In [ ]:
playerHeading("Rick Nash")

'<h1> Rick Nash </h1>'

In [ ]:
playerHeading("Henrik Lundqvist")

'<h1> Henrik Lundqvist </h1>'

This may seem pretty simplistic, but conceptually, **this is** how websites work.  The website takes in an input from the user when the user visits a URL.  And then the website returns different HTML to the user's browser, which the browser then interprets.

Now that we understand this idea, we can move onto building some real websites.

### Dynamic Websites in Flask

In Flask, we can generate this with the following:

```python
from flask import Flask
app = Flask(__name__)

@app.route('/players/<name>')
def show_player(name):
    return f'Welcome to the page of: <h1> {name} </h1>'

app.run(debug = True)
```

Now when we visit the page `/players/henrik` the web app will return the string `welcome to the webpage of <h1> henrik </h1>`.

Give it a shot.

Copy and paste the code above into a file called `app.py`, and then run `python app.py`.

<img src="https://github.com/apis-jigsaw/building-websites/blob/main/henrik.png?raw=1" width="50%">

<img src="https://github.com/apis-jigsaw/building-websites/blob/main/claude.png?raw=1" width="50%">

### Breaking it Down

Ok, now let's take some time to better understand the code that accomplished this.  Once again, here is our code:

```python
from flask import Flask
app = Flask(__name__)

@app.route('/players/<name>')
def show_player(name):
    return f'Welcome to the page of: <h1> {name} </h1>'

app.run(debug = True)
```

The first thing to notice is the argument to `@app.route`, which is `/players/<name>`.

Those brackets surrounding `player_name` indicate that the user can go to any url that has the pattern `/players/anything_else`.  And whatever the user types in after that second slash is passed to our `show_player` function as the `name` argument.  Then we use that argument to return the string below.  

You can see this code live, by going to the [server.py file in the following link](https://glitch.com/embed/#!/embed/fanatical-magpie?path=server.py?path=server.py:7:0).  
  
Ok, let's start from the beginning to make sure that we understand:
  
  1. The user types in a url, or more likely clicks on a link to a url that specifies a player.  Like `www.espn.com/players/giroux`.  
  2. This url comes into our `app.py` file, and our program sees that it matches the route:
  `@app.route('/players/<player_name>')`, as it matches the pattern of  `/players` plus slash `something_else`.
  3. That `something_else` gets passed to the function as an argument, here `player_name` and we can use that data to combine it with some HTML.


Try it out by visiting different urls and seeing how the website's output changes.

### Summary

In this lesson, we saw that the HTML behind many webpages are quite similar.  In building a website, we can generate slightly different webpages by taking in an input from the user, and having a function that returns a slightly different string of HTML.  

```python
def playerHeading(playerName):
    return f"<h1> {playerName} </h1>"
```

Then we saw that we can use this same concept to build a dynamic website with the following code:

```python
from flask import Flask
app = Flask(__name__)

@app.route('/players/<name>')
def show_player(name):
    return f'Welcome to the page of: <h1> {name} </h1>'
```

With this code, as we visit new urls of `/players/<name>`, the web application responds with different strings.